# Instrumental Variables

In the previous section, we saw how linear regression can provide unbiased estimates under certain conditions. Multiple regression was used to account for omitted variable bias. However, even if a model is specified correctly and all of the relevant covariates are included, there are still a few situations that can lead to biased and/or inconsistent estimates. The most common case is when a variable exists that is known to be an important covariate, but is not observable. However, there are three additional situations where bias is introduced:

1. Imprecise measurement of the independent variable
2. Sample selection bias
3. Simultaneous causality

Biased estimates are a threat to the **internal validtiy** of an analysis, so ideally we want a method to account for these remaining issues. Each of these remaining problems have one thing in common: they result in one or more of the included covariates $X$ being correlated with the error term $\epsilon$. Said differently, a regressor is no longer **exogenous**, but is **endogeneous**. Instrumental variables is a technique for handling this general class of problems.

Intuitively, every regressor can be broken down into two components: the part that is correlated with the error term and the part that is not. If you could isolate the part that is uncorrelated with the error term, then you are back in the happy situation of being able to compute unbiased estimates. A valid instrumental variable does exactly that: it helps to isolate the variation in the endogenous regressor X that is uncorrelated with the error term. An instrumental variable $Z$ is any variable that satisfies the following conditions:

1. Relevance -- $corr(Z_i, X_i) \neq 0$
2. Exogeneity -- $corr(Z_i, \epsilon_i) = 0$

In words, the instrument must be correlated with the endogeneous regressor and it cannot be correlated with the error term. If we can identify a valid instrument, then we can achieve consistent estimates using the **two stage least squares** estimator.


## Instrumental Variables and Class Size

Consider our running example of the effect of class size on student achievement. Although there are certainly more, one important factor is the existence of outside learning opportunities. However, it is not immediately obvious how such a factor could be quantified and measured. Excluding such information would lead to biased estimates, but taking an instrumental variables approach provides us with the unbiased estimate we desire. 

Ideally, we can identify a variable that is correlated with class size (relevant) but uncorrelated with the presence of outside learning opportunities (exogenous) and other hard-to-measure factors. Another way to think about such a variable is as something that induces a random change (exogenously) in class size. Hoxby (2000) studied exactly this problem and suggested using potential enrollment as an instrument. Since birth dates/months are random, potential enrollment should be random as well. However, potential enrollment could be endogenous because parents may move to districts with better schools. In this situation, potential enrollment is correlated with unobserved factors about the school that would also effect test scores. Instead, Hoxby (2000) use the difference between potential enrollment and its long-term trend as the instrument.

### Exogenous, but Irrelvant

First, let's consider an instrument that is exogenous, i.e. uncorrelated with the error term, but plays no part in determining test score outcomes (does not satisify the relevance criteria). In this example, assume that we have data on whether or not a student's parent drives a white car.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.tools.tools import add_constant
from scipy.stats import ttest_1samp, ttest_ind
from scipy.linalg import cholesky
from sklearn.preprocessing import minmax_scale, scale

%matplotlib inline

In [2]:
sample_size = 1000
treatment = np.random.choice([0, 1], sample_size)

treatment_df = pd.DataFrame()

# White car is totally random and unrelated to anything else
treatment_df["white_car"] = np.random.choice([0, 1], sample_size)

# Construct treatment and outside learning opportunities to be correlated
corr_mat = np.array([
    [1.0, 0.6],
    [0.6, 1.00]
])

upper_cholesky = cholesky(corr_mat)
random_data = np.random.normal(0, 1, size=(sample_size, 2))
transformed_data = random_data @ upper_cholesky

outside_learning_opp = minmax_scale(transformed_data[:, 0], feature_range=(0,1))
outside_learning_opp = [1 if s > 0.5 else 0 for s in outside_learning_opp]

treatment = minmax_scale(transformed_data[:, 1] ,feature_range=(0,1))
# Ensure balanced classes
treatment = [1 if s > np.median(treatment) else 0 for s in treatment]

# Build up our dataset
treatment_df["treated"] = treatment
treatment_df["outside_learning"] = outside_learning_opp

treatment_effect = 10
outside_learning_opp_effect = 3
treatment_df["score"] = (72 + 
                         treatment_effect*treatment_df["treated"] +
                         outside_learning_opp_effect*treatment_df["outside_learning"] + 
                         np.random.normal(loc=0.0, scale=5.0, size=sample_size))

In [3]:
model = smf.ols(formula="score ~ 1 + treatment", data=treatment_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.520
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     1083.
Date:                Sat, 29 Jun 2019   Prob (F-statistic):          1.92e-161
Time:                        16:23:11   Log-Likelihood:                -3055.5
No. Observations:                1000   AIC:                             6115.
Df Residuals:                     998   BIC:                             6125.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     73.3516      0.230    318.933      0.000      72.900      73.803
treatment     10.7043      0.325     32.910      0.000      10.066      11.343
==============================================================================
Omnibus:                        2.971   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.226   Jarque-Bera (JB):                2.609
Skew:                          -0.037   Prob(JB):                        0.271
Kurtosis:                       2.761   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We constructed the data such that reducing class size improves test scores by 10 points and having outside learning opportunities improves test scores by 3 points. We also constructed the data so that treatment and outside learning opportunities are highly correlated. For the sake of illustration, we assume that outside learning opportunities are not directly observable. The model that was fit without including outside learning opportunities has a (upward) biased estiate of the effect of class size on performance.

Let's see if this bias disappears when we use car color as an instrument for outside learning opportunities. The IV estimate of the treatment effect can be achieved by fitting two models via least squares. In the first stage, we regress the treatment variable on our instrument.

In [4]:
first_stage_model = smf.ols(formula="treatment ~ 1 + white_car", data=treatment_df)
first_stage_result = first_stage_model.fit()
first_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              treatment   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.1957
Date:                Sat, 29 Jun 2019   Prob (F-statistic):              0.658
Time:                        16:23:14   Log-Likelihood:                -725.69
No. Observations:                1000   AIC:                             1455.
Df Residuals:                     998   BIC:                             1465.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5071      0.023     22.499      0.000       0.463       0.551
white_car     -0.0140      0.032     -0.442      0.658      -0.076       0.048
==============================================================================
Omnibus:                     3765.203   Durbin-Watson:                   2.058
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              166.536
Skew:                           0.000   Prob(JB):                     6.87e-37
Kurtosis:                       1.001   Cond. No.                         2.63
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This regression summary is already a little fishy. First, the model explains very little of the variance in the treatment variable. Second, one piece of advice (albeit folklore) is that for a good instrument, the first stage F-statistic should be greater than 10. Otherwise, the instrument is likely that the instrument is too weak to be useful. For the sake of illustration, we will continue with the second stage. For the second stage, we first compute the predicted values for treatment using the estimated coefficients from the first stage. Then, we regress the test scores outcome variable on the predicted treatment variables. 

In [5]:
predicted_values = first_stage_result.predict(treatment_df)
treatment_df["predicted_treatment"] = predicted_values

second_stage_model = smf.ols(formula="score ~ 1 + predicted_treatment", data=treatment_df)
second_stage_result = second_stage_model.fit()
second_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.08460
Date:                Sat, 29 Jun 2019   Prob (F-statistic):              0.771
Time:                        16:23:16   Log-Likelihood:                -3422.9
No. Observations:                1000   AIC:                             6850.
Df Residuals:                     998   BIC:                             6860.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              83.5821     16.774      4.983      0.000      50.666     116.498
predicted_treatment    -9.7567     33.544     -0.291      0.771     -75.582      56.068
==============================================================================
Omnibus:                       19.636   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               11.219
Skew:                          -0.049   Prob(JB):                      0.00366
Kurtosis:                       2.490   Cond. No.                         179.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The new estimate for the effect of treatment of test scores is effectively zero. However, we should not put much stock in this estimate since we constructed the instrument to violate the relevance assumption. In practice, we would never have chosen car color as an instrument since there is no theoretical reason why such information would have made for a valid instrument. Secondly, after seeing the results of the first stage, we would not have had any evidence that that proposed instrument was strong enough to yield valid results.

### Relevant, but Endogenous

We now consider what happens in the opposite case: the instrument is relevant (correlated with treatment), but not exogenous. 

In [12]:
# Construct correlation matrix such that:
#  - OLO and Treatment are correlated
#  - Treatment and % Change in Potential Enrollment are correlated
#  - OLO and % Change in Potential Enrollment are correlated (endogenous)
corr_mat = np.array([
    [1.0, 0.6, 0.4],
    [0.6, 1.0, 0.6],
    [0.4, 0.6, 1.0]
])

sample_size = 1000
upper_cholesky = cholesky(corr_mat)
random_data = np.random.normal(0, 1, size=(sample_size, 3))
transformed_data = random_data @ upper_cholesky

# Boolean for outside learning opportunity
outside_learning_opp = minmax_scale(transformed_data[:, 0], feature_range=(0,1))
outside_learning_opp = [1 if s > 0.5 else 0 for s in outside_learning_opp]

# Boolean for treatment
treatment = minmax_scale(transformed_data[:, 1] ,feature_range=(0,1))
treatment = [1 if s > np.median(treatment) else 0 for s in treatment]

change_enroll = minmax_scale(transformed_data[:, 2] ,feature_range=(0, 1))

# Build up our dataset
treatment_df = pd.DataFrame()
treatment_df["treatment"] = treatment
treatment_df["outside_learning"] = outside_learning_opp
treatment_df["change_enroll"] = change_enroll

treatment_effect = 10
outside_learning_opp_effect = 3
treatment_df["score"] = (72 + 
                         treatment_effect*treatment_df["treatment"] +
                         outside_learning_opp_effect*treatment_df["outside_learning"] + 
                         np.random.normal(loc=0.0, scale=1.0, size=sample_size))

In [14]:
# Sanity Check: Including all relevant information leads to unbiased estimates
model = smf.ols(formula="score ~ 1 + treatment + outside_learning_opp", data=treatment_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                 1.692e+04
Date:                Sat, 29 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:31:55   Log-Likelihood:                -1411.8
No. Observations:                1000   AIC:                             2830.
Df Residuals:                     997   BIC:                             2844.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               72.0350      0.051   1422.513      0.000      71.936      72.134
treatment               10.0420      0.069    145.904      0.000       9.907      10.177
outside_learning_opp     2.9856      0.069     43.144      0.000       2.850       3.121
==============================================================================
Omnibus:                        1.570   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.456   Jarque-Bera (JB):                1.568
Skew:                          -0.047   Prob(JB):                        0.456
Kurtosis:                       2.830   Cond. No.                         3.40
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# Sanity Check: No IV leads to biased estimates
model = smf.ols(formula="score ~ 1 + treatment", data=treatment_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.918
Method:                 Least Squares   F-statistic:                 1.117e+04
Date:                Sat, 29 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:34:46   Log-Likelihood:                -1938.5
No. Observations:                1000   AIC:                             3881.
Df Residuals:                     998   BIC:                             3891.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     73.0800      0.075    971.008      0.000      72.932      73.228
treatment     11.2482      0.106    105.680      0.000      11.039      11.457
==============================================================================
Omnibus:                       20.211   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               11.669
Skew:                          -0.068   Prob(JB):                      0.00292
Kurtosis:                       2.489   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The estimated ATE is clearly biased upwards by not including outside learning opportunities in the model.

In [21]:
first_stage_model = smf.ols(formula="treatment ~ 1 + change_enroll", data=treatment_df)
first_stage_result = first_stage_model.fit()
first_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              treatment   R-squared:                       0.235
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     307.2
Date:                Sat, 29 Jun 2019   Prob (F-statistic):           3.56e-60
Time:                        16:34:57   Log-Likelihood:                -591.60
No. Observations:                1000   AIC:                             1187.
Df Residuals:                     998   BIC:                             1197.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.2274      0.044     -5.199      0.000      -0.313      -0.142
change_enroll     1.4381      0.082     17.528      0.000       1.277       1.599
==============================================================================
Omnibus:                      578.583   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.972
Skew:                           0.025   Prob(JB):                     4.25e-13
Kurtosis:                       1.832   Cond. No.                         7.48
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Things look much less fishy this time! Our new instrument explains some of the variance in treatment. Additionally, we are well over the suggested F-statistic of 10, which suggests we might have some luck with this instrument.

In [23]:
predicted_values = first_stage_result.predict(treatment_df)
treatment_df["predicted_treatment"] = predicted_values

second_stage_model = smf.ols(formula="score ~ 1 + predicted_treatment", data=treatment_df)
second_stage_result = second_stage_model.fit()
second_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.244
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     322.7
Date:                Sat, 29 Jun 2019   Prob (F-statistic):           9.82e-63
Time:                        16:36:57   Log-Likelihood:                -3048.7
No. Observations:                1000   AIC:                             6101.
Df Residuals:                     998   BIC:                             6111.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              72.7235      0.370    196.531      0.000      71.997      73.450
predicted_treatment    11.9611      0.666     17.964      0.000      10.654      13.268
==============================================================================
Omnibus:                      142.596   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.217
Skew:                          -0.013   Prob(JB):                     2.25e-08
Kurtosis:                       2.081   Cond. No.                         5.20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Hmm... using the estimated treatment variable fom the first stage does not seem to have really altered the estimated ATE. If this were a valid instrument, we would expect that the estimate would be close to the true value of 10 that was used to construct the data.

In [24]:
iv_model = IV2SLS(treatment_df["score"],
                  add_constant(treatment_df["treatment"]),
                  add_constant(treatment_df["change_enroll"]))
iv_results = iv_model.fit()
iv_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                  score   R-squared:                       0.914
Model:                         IV2SLS   Adj. R-squared:                  0.914
Method:                     Two Stage   F-statistic:                     2845.
                        Least Squares   Prob (F-statistic):          1.97e-294
Date:                Sat, 29 Jun 2019                                         
Time:                        16:37:38                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         72.7235      0.125    583.525      0.000      72.479      72.968
treatment     11.9611      0.224     53.337      0.000      11.521      12.401
==============================================================================
Omnibus:                        1.012   Durbin-Watson:                   1.951
Prob(Omnibus):                  0.603   Jarque-Bera (JB):                1.046
Skew:                          -0.007   Prob(JB):                        0.593
Kurtosis:                       2.842   Cond. No.                         2.62
==============================================================================
"""

### Valid, but Weak Instrument

In [37]:
# Construct correlation matrix such that:
#  - OLO and Treatment are correlated
#  - Treatment and % Change in Potential Enrollment are correlated
#  - OLO and % Change in Potential Enrollment are uncorrelated
corr_mat = np.array([
    [1.0, 0.6, 0.0],
    [0.6, 1.0, 0.1],
    [0.0, 0.1, 1.0]
])
sample_size = 1000
upper_cholesky = cholesky(corr_mat)
random_data = np.random.normal(0, 1, size=(sample_size, 3))
transformed_data = random_data @ upper_cholesky

# Boolean for outside learning opportunity
outside_learning_opp = minmax_scale(transformed_data[:, 0], feature_range=(0,1))
outside_learning_opp = [1 if s > 0.5 else 0 for s in outside_learning_opp]

# Boolean for treatment
treatment = minmax_scale(transformed_data[:, 1] ,feature_range=(0,1))
treatment = [1 if s > np.median(treatment) else 0 for s in treatment]

change_enroll = minmax_scale(transformed_data[:, 2] ,feature_range=(0, 1))


# Build up our dataset
treatment_df = pd.DataFrame()
treatment_df["treatment"] = treatment
treatment_df["outside_learning"] = outside_learning_opp
treatment_df["change_enroll"] = change_enroll

treatment_effect = 10
outside_learning_opp_effect = 3
treatment_df["score"] = (72 + 
                         treatment_effect*treatment_df["treatment"] +
                         outside_learning_opp_effect*treatment_df["outside_learning"] + 
                         np.random.normal(loc=0.0, scale=1.0, size=sample_size))

In [38]:
model = smf.ols(formula="score ~ 1 + treatment + outside_learning_opp", data=treatment_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                 1.672e+04
Date:                Sat, 29 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:47:44   Log-Likelihood:                -1416.4
No. Observations:                1000   AIC:                             2839.
Df Residuals:                     997   BIC:                             2854.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               71.9827      0.048   1485.048      0.000      71.888      72.078
treatment                9.9549      0.069    144.226      0.000       9.819      10.090
outside_learning_opp     3.1000      0.069     44.848      0.000       2.964       3.236
==============================================================================
Omnibus:                        3.143   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.208   Jarque-Bera (JB):                2.674
Skew:                          -0.014   Prob(JB):                        0.263
Kurtosis:                       2.748   Cond. No.                         3.28
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
# Clearly mis-specified model
model = smf.ols(formula="score ~ 1 + treatment", data=treatment_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                 1.043e+04
Date:                Sat, 29 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:47:44   Log-Likelihood:                -1968.6
No. Observations:                1000   AIC:                             3941.
Df Residuals:                     998   BIC:                             3951.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.8259      0.078    938.877      0.000      72.674      72.978
treatment     11.2011      0.110    102.110      0.000      10.986      11.416
==============================================================================
Omnibus:                       28.759   Durbin-Watson:                   2.061
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               14.274
Skew:                           0.005   Prob(JB):                     0.000795
Kurtosis:                       2.415   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
first_stage_model = smf.ols(formula="treatment ~ 1 + change_enroll", data=treatment_df)
first_stage_result = first_stage_model.fit()
first_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              treatment   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     6.256
Date:                Sat, 29 Jun 2019   Prob (F-statistic):             0.0125
Time:                        16:47:46   Log-Likelihood:                -722.67
No. Observations:                1000   AIC:                             1449.
Df Residuals:                     998   BIC:                             1459.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.3886      0.047      8.227      0.000       0.296       0.481
change_enroll     0.2582      0.103      2.501      0.013       0.056       0.461
==============================================================================
Omnibus:                     3820.911   Durbin-Watson:                   1.951
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              162.521
Skew:                          -0.000   Prob(JB):                     5.12e-36
Kurtosis:                       1.025   Cond. No.                         7.78
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The chosen instrument seems to explain very little of the variance in treatment. This is reflected in the F-statistic of 5.5, which is below the suggested threshold of 10. Since we have a very weak instrument, the assumption of a normal distribution for the sampling distribution of the IV estimator is no longer valid regardless of the sample size. Therefore, we have no reason to belive that the IV estimate will be any less biased than the estimate from standard OLS.

In [43]:
predicted_values = first_stage_result.predict(treatment_df)
treatment_df["predicted_treatment"] = predicted_values

second_stage_model = smf.ols(formula="score ~ 1 + predicted_treatment", data=treatment_df)
second_stage_result = second_stage_model.fit()
second_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     5.551
Date:                Sat, 29 Jun 2019   Prob (F-statistic):             0.0187
Time:                        16:52:50   Log-Likelihood:                -3184.7
No. Observations:                1000   AIC:                             6373.
Df Residuals:                     998   BIC:                             6383.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              72.9027      2.352     30.997      0.000      68.287      77.518
predicted_treatment    11.0476      4.689      2.356      0.019       1.846      20.249
==============================================================================
Omnibus:                     4887.082   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              119.380
Skew:                           0.021   Prob(JB):                     1.19e-26
Kurtosis:                       1.308   Cond. No.                         31.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Our expectation based on how weak the instrument is seems to have been born out in the second stage regression. The estimate is closer to the true vaule of 10, but barely. The IV estimate seems to be biased in the same direction as or original OLS estimate.

In [44]:
iv_model = IV2SLS(treatment_df["score"],
                  add_constant(treatment_df["treatment"]),
                  add_constant(treatment_df["change_enroll"]))
iv_results = iv_model.fit()
iv_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                  score   R-squared:                       0.912
Model:                         IV2SLS   Adj. R-squared:                  0.912
Method:                     Two Stage   F-statistic:                     63.06
                        Least Squares   Prob (F-statistic):           5.37e-15
Date:                Sat, 29 Jun 2019                                         
Time:                        16:53:40                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         72.9027      0.698    104.484      0.000      71.533      74.272
treatment     11.0476      1.391      7.941      0.000       8.318      13.778
==============================================================================
Omnibus:                       42.830   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.465
Skew:                           0.014   Prob(JB):                     9.78e-05
Kurtosis:                       2.335   Cond. No.                         2.62
==============================================================================
"""

### Valid Instrument

Consider the population of the school district as a potential instrument. The number of classrooms for a particular grade is typically fixed (at least in the short run), therefore a large district is likely to have more potential students, and therefore larger classrooms on average. District population is therefore correlated with 

In [25]:
# Construct correlation matrix such that:
#  - OLO and Treatment are correlated
#  - Treatment and % Change in Potential Enrollment are correlated
#  - OLO and % Change in Potential Enrollment are uncorrelated
corr_mat = np.array([
    [1.0, 0.6, 0.0],
    [0.6, 1.0, 0.6],
    [0.0, 0.6, 1.0]
])
sample_size = 1000
upper_cholesky = cholesky(corr_mat)
random_data = np.random.normal(0, 1, size=(sample_size, 3))
transformed_data = random_data @ upper_cholesky

# Boolean for outside learning opportunity
outside_learning_opp = minmax_scale(transformed_data[:, 0], feature_range=(0,1))
outside_learning_opp = [1 if s > 0.5 else 0 for s in outside_learning_opp]

# Boolean for treatment
treatment = minmax_scale(transformed_data[:, 1] ,feature_range=(0,1))
treatment = [1 if s > np.median(treatment) else 0 for s in treatment]

change_enroll = minmax_scale(transformed_data[:, 2] ,feature_range=(0, 1))


# Build up our dataset
treatment_df = pd.DataFrame()
treatment_df["treatment"] = treatment
treatment_df["outside_learning"] = outside_learning_opp
treatment_df["change_enroll"] = change_enroll

treatment_effect = 10
outside_learning_opp_effect = 3
treatment_df["score"] = (72 + 
                         treatment_effect*treatment_df["treatment"] +
                         outside_learning_opp_effect*treatment_df["outside_learning"] + 
                         np.random.normal(loc=0.0, scale=1.0, size=sample_size))

In [26]:
model = smf.ols(formula="score ~ 1 + treatment + outside_learning_opp", data=treatment_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                 1.613e+04
Date:                Sat, 29 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:37:55   Log-Likelihood:                -1422.8
No. Observations:                1000   AIC:                             2852.
Df Residuals:                     997   BIC:                             2866.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               72.0371      0.047   1519.718      0.000      71.944      72.130
treatment                9.9573      0.069    144.871      0.000       9.822      10.092
outside_learning_opp     3.0339      0.070     43.249      0.000       2.896       3.172
==============================================================================
Omnibus:                        2.032   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.362   Jarque-Bera (JB):                1.944
Skew:                           0.106   Prob(JB):                        0.378
Kurtosis:                       3.041   Cond. No.                         3.18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
# Clearly mis-specified model
model = smf.ols(formula="score ~ 1 + treatment", data=treatment_df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                 1.058e+04
Date:                Sat, 29 Jun 2019   Prob (F-statistic):               0.00
Time:                        16:37:55   Log-Likelihood:                -1951.0
No. Observations:                1000   AIC:                             3906.
Df Residuals:                     998   BIC:                             3916.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.6863      0.076    953.733      0.000      72.537      72.836
treatment     11.0859      0.108    102.856      0.000      10.874      11.297
==============================================================================
Omnibus:                       15.785   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               11.844
Skew:                           0.165   Prob(JB):                      0.00268
Kurtosis:                       2.581   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
first_stage_model = smf.ols(formula="treatment ~ 1 + change_enroll", data=treatment_df)
first_stage_result = first_stage_model.fit()
first_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              treatment   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     326.5
Date:                Sat, 29 Jun 2019   Prob (F-statistic):           2.29e-63
Time:                        16:37:57   Log-Likelihood:                -584.26
No. Observations:                1000   AIC:                             1173.
Df Residuals:                     998   BIC:                             1182.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.5703      0.061     -9.380      0.000      -0.690      -0.451
change_enroll     1.8815      0.104     18.071      0.000       1.677       2.086
==============================================================================
Omnibus:                      629.323   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.072
Skew:                           0.021   Prob(JB):                     2.45e-13
Kurtosis:                       1.820   Cond. No.                         10.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Things look much less fishy this time! Our new instrument explains a lot of the variance in treatment. Additionally, we are well over the suggested F-statistic of 10, which suggests we might have some luck with this instrument.

In [29]:
predicted_values = first_stage_result.predict(treatment_df)
treatment_df["predicted_treatment"] = predicted_values

second_stage_model = smf.ols(formula="score ~ 1 + predicted_treatment", data=treatment_df)
second_stage_result = second_stage_model.fit()
second_stage_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.188
Method:                 Least Squares   F-statistic:                     232.4
Date:                Sat, 29 Jun 2019   Prob (F-statistic):           2.52e-47
Time:                        16:37:59   Log-Likelihood:                -3071.9
No. Observations:                1000   AIC:                             6148.
Df Residuals:                     998   BIC:                             6158.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              73.1541      0.372    196.800      0.000      72.425      73.884
predicted_treatment    10.1502      0.666     15.243      0.000       8.844      11.457
==============================================================================
Omnibus:                      239.729   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               45.538
Skew:                           0.114   Prob(JB):                     1.29e-10
Kurtosis:                       1.980   Cond. No.                         5.09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Note that these standard errors are wrong and show what happens when correct ones are calculated

In [30]:
iv_model = IV2SLS(treatment_df["score"],
                  add_constant(treatment_df["treatment"]),
                  add_constant(treatment_df["change_enroll"]))
iv_results = iv_model.fit()
iv_results.summary()

/home/zachduey/Documents/metrics/env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                  score   R-squared:                       0.907
Model:                         IV2SLS   Adj. R-squared:                  0.907
Method:                     Two Stage   F-statistic:                     2033.
                        Least Squares   Prob (F-statistic):          5.58e-243
Date:                Sat, 29 Jun 2019                                         
Time:                        16:38:01                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         73.1541      0.126    582.111      0.000      72.908      73.401
treatment     10.1502      0.225     45.088      0.000       9.708      10.592
==============================================================================
Omnibus:                       80.576   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.657
Skew:                           0.272   Prob(JB):                     1.10e-08
Kurtosis:                       2.236   Cond. No.                         2.62
==============================================================================
"""

Note: Those standard errors seem suspect to me. I would have expected that they would be larger than in the manual two stage estimates. Ran the same IV regression in R and the reported standard error for the coefficient on treatment is 0.10563